## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%load_ext autoreload

%autoreload 

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,51.8,58,75,5.75,broken clouds
1,1,Avarua,CK,-21.2078,-159.7750,75.2,94,75,19.57,light rain
2,2,Albany,US,42.6001,-73.9662,41.0,34,100,3.00,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,44.6,87,75,5.75,broken clouds
4,4,Barrow,US,71.2906,-156.7887,19.4,73,1,6.91,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,75.20,94,75,19.57,light rain
5,5,Mudon,MM,16.2578,97.7164,80.67,79,65,4.56,broken clouds
7,7,Georgetown,MY,5.4112,100.3354,77.00,94,40,1.14,scattered clouds
8,8,Kapaa,US,22.0752,-159.3190,78.01,83,40,9.22,scattered clouds
12,12,Hilo,US,19.7297,-155.0900,80.60,57,40,9.22,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
preferred_cities_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,75.20,94,75,19.57,light rain
5,5,Mudon,MM,16.2578,97.7164,80.67,79,65,4.56,broken clouds
7,7,Georgetown,MY,5.4112,100.3354,77.00,94,40,1.14,scattered clouds
8,8,Kapaa,US,22.0752,-159.3190,78.01,83,40,9.22,scattered clouds
12,12,Hilo,US,19.7297,-155.0900,80.60,57,40,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
663,663,Tessalit,ML,20.1986,1.0114,87.60,6,1,9.37,clear sky
665,665,Salinopolis,BR,-0.6136,-47.3561,77.50,87,100,6.02,overcast clouds
670,670,Vilhena,BR,-12.7406,-60.1458,86.00,62,40,8.05,scattered clouds
672,672,Trat,TH,12.5000,102.5000,82.99,99,92,1.90,overcast clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,75.20,light rain,-21.2078,-159.7750,
5,Mudon,MM,80.67,broken clouds,16.2578,97.7164,
7,Georgetown,MY,77.00,scattered clouds,5.4112,100.3354,
8,Kapaa,US,78.01,scattered clouds,22.0752,-159.3190,
12,Hilo,US,80.60,scattered clouds,19.7297,-155.0900,
17,Mecca,SA,86.27,overcast clouds,21.4267,39.8261,
18,Rikitea,PF,77.86,light rain,-23.1203,-134.9692,
24,Tangkak,MY,78.80,few clouds,2.2673,102.5453,
27,Puerto Leguizamo,CO,82.85,scattered clouds,-0.1934,-74.7819,
36,Kloulklubed,PW,75.20,scattered clouds,7.0419,134.2556,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [54]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,75.20,light rain,-21.2078,-159.7750,Paradise Inn
5,Mudon,MM,80.67,broken clouds,16.2578,97.7164,Motel Mudon
7,Georgetown,MY,77.00,scattered clouds,5.4112,100.3354,Cititel Penang
8,Kapaa,US,78.01,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
12,Hilo,US,80.60,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [55]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,75.20,light rain,-21.2078,-159.7750,Paradise Inn
5,Mudon,MM,80.67,broken clouds,16.2578,97.7164,Motel Mudon
7,Georgetown,MY,77.00,scattered clouds,5.4112,100.3354,Cititel Penang
8,Kapaa,US,78.01,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
12,Hilo,US,80.60,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(25.0, 35.0), zoom_level=1.25)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))